#### Draw R2 across ROI and attention condition

In [ ]:
# Imports
import os
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# Figure imports
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.express as px
from plot_utils import plotly_template

# Define parameters
subjects = ['sub-001', 'sub-002', 'sub-003', 'sub-004',
            'sub-005', 'sub-006', 'sub-007', 'sub-008']
subjects_plot = ['sub-001', 'sub-002', 'sub-003', 'sub-004',
                 'sub-005', 'sub-006', 'sub-007', 'sub-008', 'group']
tasks = ['FullScreen', 'FullScreenAttendFix', 'FullScreenAttendBar']
rois = ['V1', 'V2', 'V3', 'V3AB', 'hMT+', 'LO',
        'VO', 'iIPS', 'sIPS', 'iPCS', 'sPCS', 'mPCS']

# Define folders
base_dir = '/home/mszinte/disks/meso_S/data/gaze_prf'
bids_dir = "{}".format(base_dir)
pp_dir = "{}/derivatives/pp_data".format(base_dir)

# settings
n_ecc_bins = 10
best_voxels_num = 250
type_analyses = ['','_best{}'.format(best_voxels_num)]

# General figure settings
template_specs = dict(  axes_color="rgba(0, 0, 0, 1)",
                        axes_width=2,
                        axes_font_size=13,
                        bg_col="rgba(255, 255, 255, 1)",
                        font='Arial',
                        title_font_size=15,
                        plot_width=1.5)
fig_template = plotly_template(template_specs)

ab_line_col = "rgba(0, 0, 0, 1)"
af_line_col = "rgba(157, 157, 157, 1)"
ab_area_col = "rgba(0, 0, 0, 0.2)"
af_area_col = "rgba(157, 157, 157, 0.2)"

In [ ]:
# Subplot settings
rows, cols = 1, 1
fig_height, fig_width = 800,400
column_widths,row_heights = [1],[1]
sb_specs = [[{}]]
hovertemplate_task = 'ROI: %{y}<br>' + 'R2: %{x:1.2f}'

for type_analysis in type_analyses:
    if type_analysis == '_best{}'.format(best_voxels_num):
        xaxis_range = [0,0.8]
        xaxis_dtick = 0.2
    else:
        xaxis_range = [0,0.4]
        xaxis_dtick = 0.1
    x_gap_line = 0.1*xaxis_range[1]
    x_gap_asterisk = 0.15*xaxis_range[1]
    
    for subject in subjects_plot:
        subplot_titles = ['<b>Attention effect</b> ({})'.format(subject)]

        fig = make_subplots(rows=rows, cols=cols, specs=sb_specs, print_grid=False, vertical_spacing=0.15, horizontal_spacing=0.05, 
                        column_widths=column_widths, row_heights=row_heights,  subplot_titles=subplot_titles)

        # Get Data
        tsv_dir = '{}/{}/prf/tsv'.format(pp_dir, subject)
        af_df = pd.read_csv("{}/{}_task-FullScreenAttendFix_prf_threshold_par{}.tsv".format(tsv_dir,subject,type_analysis), sep="\t")
        ab_df = pd.read_csv("{}/{}_task-FullScreenAttendBar_prf_threshold_par{}.tsv".format(tsv_dir,subject,type_analysis), sep="\t")
        af_df.roi = pd.Categorical(af_df.roi,categories=rois)
        ab_df.roi = pd.Categorical(af_df.roi,categories=rois)

        x_af, x_ab = af_df.groupby('roi').r2.mean(), ab_df.groupby('roi').r2.mean()
        if subject == 'group': 
            x_af_eb = af_df.groupby(['roi','subject']).r2.mean().reset_index(name='r2').groupby('roi').r2.std()/np.sqrt(len(subjects)-1)
            x_ab_eb = ab_df.groupby(['roi','subject']).r2.mean().reset_index(name='r2').groupby('roi').r2.std()/np.sqrt(len(subjects)-1)
        else: 
            x_af_eb, x_ab_eb = af_df.groupby('roi').r2.std(), ab_df.groupby('roi').r2.std()


        # bar plot
        fig.append_trace(go.Bar(y=rois, x=x_ab, orientation='h',
                                showlegend=True, legendgroup='attend-bar', name='<i>attend-bar<i>',
                                marker_color=ab_line_col, hovertemplate=hovertemplate_task,
                                error_x_type='data', error_x_symmetric=False, error_x_array=x_ab_eb, error_x_arrayminus=x_ab_eb),row=1, col=1)

        fig.append_trace(go.Bar(y=rois, x=x_af, orientation='h',
                                showlegend=True, legendgroup='attend-fix', name='<i>attend-fix<i>', 
                                marker_color=af_line_col, hovertemplate=hovertemplate_task,
                                error_x_type='data', error_x_symmetric=False, error_x_array=x_af_eb, error_x_arrayminus=x_af_eb),row=1, col=1)


        # get Stats
        if subject == 'group': 
            stats_df = pd.read_csv("{}/group_task-FullScreen_prf_threshold_stats{}.tsv".format(tsv_dir,type_analysis), sep="\t")
            p_flag = stats_df.p_val_unilateral<0.05
            fig.append_trace(go.Scatter(x=np.max((x_ab,x_af),axis=0)[p_flag]+x_gap_line, y=np.array(rois)[p_flag],
                                        marker_symbol='line-ns-open',
                                        showlegend=False, mode='markers', marker_color=ab_line_col, hoverinfo='skip',
                                        line_width=1, opacity=1, marker_size= 15),row=1, col=1)
            fig.append_trace(go.Scatter(x=np.max((x_ab,x_af),axis=0)[p_flag]+x_gap_asterisk, y=np.array(rois)[p_flag],
                                        marker_symbol='asterisk-open',
                                        showlegend=False, mode='markers', marker_color=ab_line_col, hoverinfo='skip',
                                        line_width=1, opacity=1, marker_size= 10),row=1, col=1)

        # set axis
        for row in np.arange(rows):
            for col in np.arange(cols):
                fig.update_xaxes(visible=True, ticklen=8, linewidth=template_specs['axes_width'], row=row+1, col=col+1)
                fig.update_yaxes(visible=True, ticklen=8, linewidth=template_specs['axes_width'], row=row+1, col=col+1, autorange='reversed')

        # set figure
        fig.layout.update(xaxis_range=xaxis_range, xaxis_title='R2', xaxis_dtick=xaxis_dtick,
                          yaxis_title='',bargap=0.30,
                          template=fig_template, width=fig_width, height=fig_height, margin_l=50, margin_r=20, margin_t=50, margin_b=70,
                          legend_yanchor='top', 
                          legend_y=0.995, legend_xanchor='left', legend_x=0.9, legend_bgcolor='rgba(255,255,255,0)', legend_tracegroupgap=1)

        # show and save figure
        fig.show(config={"displayModeBar": False})
        fig.write_image("{}/{}_attcmp{}.pdf".format(tsv_dir, subject, type_analysis))
        fig.write_html("{}/{}_attcmp{}.html".format(tsv_dir, subject, type_analysis),config={"displayModeBar": False})

#### Draw R2 across ROI and attention condition as a function of pRF eccentricity

In [ ]:
# Subplot settings

rows, cols = 3, 4
fig_height, fig_width = 1000,1000
column_widths,row_heights = [1,1,1,1],[1,1,1]
sb_specs = [[{},{},{},{}],[{},{},{},{}],[{},{},{},{}]]
hovertemplate_task = 'R2: %{y:1.2f}<br>' + 'Ecc.: %{x:1.2f} dva'
xaxis_range = [0,15]
yaxis_range = [0,0.8]

for type_analysis in type_analyses:
    if type_analysis == '_best{}'.format(best_voxels_num):
        yaxis_range = [0,0.8]
        yaxis_dtick = 0.2
    else: 
        yaxis_range = [0,0.4]
        yaxis_dtick = 0.1
    y_gap_asterisk = 0.15*yaxis_range[1]

    for subject_num, subject in enumerate(subjects_plot):
        subplot_titles = ['<b>V1</b> ({})'.format(subject), '<b>V2</b> ({})'.format(subject), '<b>V3</b> ({})'.format(subject), '<b>V3AB</b> ({})'.format(subject),
                      '<b>hMT+</b> ({})'.format(subject), '<b>LO</b> ({})'.format(subject), '<b>VO</b> ({})'.format(subject), '<b>iIPS</b> ({})'.format(subject),
                      '<b>sIPS</b> ({})'.format(subject), '<b>iPCS</b> ({})'.format(subject), '<b>sPCS</b> ({})'.format(subject), '<b>mPCS</b> ({})'.format(subject)]

        fig = make_subplots(rows=rows, cols=cols, specs=sb_specs, print_grid=False, vertical_spacing=0.15, horizontal_spacing=0.05, 
                            column_widths=column_widths, row_heights=row_heights, subplot_titles=subplot_titles)

        # Get Data
        tsv_dir = '{}/{}/prf/tsv'.format(pp_dir, subject)
        af_df = pd.read_csv("{}/{}_task-FullScreenAttendFix_prf_threshold_par{}.tsv".format(tsv_dir,subject,type_analysis), sep="\t")
        ab_df = pd.read_csv("{}/{}_task-FullScreenAttendBar_prf_threshold_par{}.tsv".format(tsv_dir,subject,type_analysis), sep="\t")
        af_df.roi = pd.Categorical(af_df.roi,categories=rois)
        ab_df.roi = pd.Categorical(af_df.roi,categories=rois)
        if subject == 'group': 
            stats_df = pd.read_csv("{}/group_task-FullScreen_prf_threshold_ecc_bin_num_stats{}.tsv".format(tsv_dir,type_analysis), sep="\t")

        rows, cols = 3, 4
        for roi_num, roi in enumerate(rois):

            # attention-bar bin
            ab_df_binned = ab_df[ab_df.roi==roi].assign(ecc_bin=pd.qcut(ab_df.ecc, n_ecc_bins))
            ab_df_bin_roi = pd.DataFrame({'roi':[roi] * n_ecc_bins,
                                          'ecc_bin_num':np.arange(0,n_ecc_bins),
                                          'r2_mean':np.array(ab_df_binned.groupby('ecc_bin').r2.mean()),
                                          'r2_std':np.array(ab_df_binned.groupby('ecc_bin').r2.std()),
                                          'ecc_mean':np.array(ab_df_binned.groupby('ecc_bin').ecc.mean()),
                                          'ecc_std':np.array(ab_df_binned.groupby('ecc_bin').ecc.std())})
            if roi_num == 0: ab_df_bin = ab_df_bin_roi
            else: ab_df_bin = pd.concat([ab_df_bin,ab_df_bin_roi], ignore_index=True)

            # attention-fix bins
            af_df_binned = af_df[af_df.roi==roi].assign(ecc_bin=pd.qcut(af_df.ecc, n_ecc_bins))
            af_df_bin_roi = pd.DataFrame({'roi':[roi] * n_ecc_bins,
                                          'ecc_bin_num':np.arange(0,n_ecc_bins),
                                          'r2_mean':np.array(af_df_binned.groupby('ecc_bin').r2.mean()),
                                          'r2_std':np.array(af_df_binned.groupby('ecc_bin').r2.std()),
                                          'ecc_mean':np.array(af_df_binned.groupby('ecc_bin').ecc.mean()),
                                          'ecc_std':np.array(af_df_binned.groupby('ecc_bin').ecc.std())})
            if roi_num == 0: af_df_bin = af_df_bin_roi
            else: af_df_bin = pd.concat([af_df_bin,af_df_bin_roi], ignore_index=True)

            # plot data
            row_num = np.ceil((roi_num+1)/cols)
            if roi_num == 0 or roi_num == 4 or roi_num == 8: col_num = 1
            elif roi_num == 1 or  roi_num == 5 or roi_num == 9: col_num = 2
            elif roi_num == 2 or  roi_num == 6 or roi_num == 10: col_num = 3
            elif roi_num == 3 or  roi_num == 7 or roi_num == 11: col_num = 4

            if roi_num == 0 :showlegend=True
            else: showlegend=False
            
            # stimulus lines
            ecc_fix = 0.8
            ecc_ver_bar = 8.9
            ecc_hor_bar = 5
            
            fig.add_shape(type="line", x0=ecc_fix, y0=yaxis_range[0], x1=ecc_fix, y1=yaxis_range[1],
                          line_color=ab_line_col, line_dash="dot", line_width=1, row=int(row_num), col=int(col_num))
            
            fig.add_shape(type="line", x0=ecc_ver_bar, y0=yaxis_range[0], x1=ecc_ver_bar, y1=yaxis_range[1],
                          line_color=ab_line_col, line_dash="dot", line_width=1, row=int(row_num), col=int(col_num))
            
            fig.add_shape(type="line", x0=ecc_hor_bar, y0=yaxis_range[0], x1=ecc_hor_bar, y1=yaxis_range[1],
                          line_color=ab_line_col, line_dash="dot", line_width=1, row=int(row_num), col=int(col_num))
            
            
            # scatter plot           
            fig.append_trace(go.Scatter(x=ab_df_bin[ab_df_bin.roi==roi].ecc_mean,
                                        y=ab_df_bin[ab_df_bin.roi==roi].r2_mean,
                                        legendgroup='attend-bar', name='<i>attend-bar<i>',
                                        showlegend=showlegend, mode='markers+lines', marker_color=ab_line_col, hovertemplate=hovertemplate_task,
                                        line_width=1, opacity=1, marker_size=6),row=int(row_num), col=int(col_num))
            # error sleaves
            if subject == 'group':
                fig.append_trace(go.Scatter(x=ab_df_bin[ab_df_bin.roi==roi].ecc_mean, 
                                            y=ab_df_bin[ab_df_bin.roi==roi].r2_mean+(ab_df_bin[ab_df_bin.roi==roi].r2_std/np.sqrt(len(subjects)-1)),
                                            mode='lines',line_width=0, fillcolor=ab_area_col, showlegend=False, legendgroup='attend-bar', 
                                            hoverinfo='skip'),row=int(row_num), col=int(col_num))
                fig.append_trace(go.Scatter(x=ab_df_bin[ab_df_bin.roi==roi].ecc_mean, 
                                            y=ab_df_bin[ab_df_bin.roi==roi].r2_mean-(ab_df_bin[ab_df_bin.roi==roi].r2_std/np.sqrt(len(subjects)-1)),
                                            mode='lines',line_width=0, fillcolor=ab_area_col, showlegend=False, legendgroup='attend-bar', 
                                            hoverinfo='skip',fill='tonexty',), row=int(row_num), col=int(col_num))
            # scatter plot
            fig.append_trace(go.Scatter(x=af_df_bin[af_df_bin.roi==roi].ecc_mean, 
                                        y=af_df_bin[af_df_bin.roi==roi].r2_mean,
                                        legendgroup='attend-fix', name='<i>attend-fix<i>',
                                        showlegend=showlegend, mode='markers+lines', marker_color=af_line_col, hovertemplate=hovertemplate_task,
                                        line_width=1, opacity=1, marker_size=6),row=int(row_num), col=int(col_num))
            
            # error sleaves
            if subject == 'group':
                fig.append_trace(go.Scatter(x=af_df_bin[af_df_bin.roi==roi].ecc_mean, 
                                            y=af_df_bin[af_df_bin.roi==roi].r2_mean+(af_df_bin[af_df_bin.roi==roi].r2_std/np.sqrt(len(subjects)-1)),
                                            mode='lines',line_width=0, fillcolor=af_area_col, showlegend=False, legendgroup='attend-fix', 
                                            hoverinfo='skip'),row=int(row_num), col=int(col_num))
                fig.append_trace(go.Scatter(x=af_df_bin[af_df_bin.roi==roi].ecc_mean, 
                                            y=af_df_bin[af_df_bin.roi==roi].r2_mean-(af_df_bin[af_df_bin.roi==roi].r2_std/np.sqrt(len(subjects)-1)),
                                            mode='lines',line_width=0, fillcolor=af_area_col, showlegend=False, legendgroup='attend-fix', 
                                            hoverinfo='skip',fill='tonexty',), row=int(row_num), col=int(col_num))
            
            # get Stats
            if subject == 'group': 
                p_flag = stats_df[stats_df.roi==roi].p_val_unilateral<0.05
                fig.append_trace(go.Scatter(x=np.mean((ab_df_bin[ab_df_bin.roi==roi].ecc_mean,af_df_bin[ab_df_bin.roi==roi].ecc_mean),axis=0)[p_flag],
                                            y=np.max((ab_df_bin[ab_df_bin.roi==roi].r2_mean,af_df_bin[ab_df_bin.roi==roi].r2_mean),axis=0)[p_flag]+y_gap_asterisk,
                                            marker_symbol='asterisk-open',
                                            showlegend=False, mode='markers', marker_color=ab_line_col, hoverinfo='skip',
                                            line_width=1, opacity=1, marker_size=6),row=int(row_num), col=int(col_num))
            
        # set axis
        for row in np.arange(rows):
            for col in np.arange(cols):
                fig.update_xaxes(visible=True, ticklen=8, linewidth=template_specs['axes_width'], row=row+1, col=col+1)
                fig.update_yaxes(visible=True, ticklen=8, linewidth=template_specs['axes_width'], row=row+1, col=col+1)

        
        # set figure 
        fig.layout.update(yaxis_range=yaxis_range,   yaxis_title='R2',   yaxis_dtick=yaxis_dtick,   yaxis2_range=yaxis_range,  yaxis2_title='R2',  yaxis2_dtick=yaxis_dtick,
                          yaxis3_range=yaxis_range,  yaxis3_title='R2',  yaxis3_dtick=yaxis_dtick,  yaxis4_range=yaxis_range,  yaxis4_title='R2',  yaxis4_dtick=yaxis_dtick,
                          yaxis5_range=yaxis_range,  yaxis5_title='R2',  yaxis5_dtick=yaxis_dtick,  yaxis6_range=yaxis_range,  yaxis6_title='R2',  yaxis6_dtick=yaxis_dtick,
                          yaxis7_range=yaxis_range,  yaxis7_title='R2',  yaxis7_dtick=yaxis_dtick,  yaxis8_range=yaxis_range,  yaxis8_title='R2',  yaxis8_dtick=yaxis_dtick,
                          yaxis9_range=yaxis_range,  yaxis9_title='R2',  yaxis9_dtick=yaxis_dtick,  yaxis10_range=yaxis_range, yaxis10_title='R2', yaxis10_dtick=yaxis_dtick,
                          yaxis11_range=yaxis_range, yaxis11_title='R2', yaxis11_dtick=yaxis_dtick, yaxis12_range=yaxis_range, yaxis12_title='R2', yaxis12_dtick=yaxis_dtick,
                          xaxis_range=xaxis_range,   xaxis_title='Eccentricity (dva)',   xaxis_dtick=5,   xaxis2_range=xaxis_range,  xaxis2_title='Eccentricity (dva)',  xaxis2_dtick=5,
                          xaxis3_range=xaxis_range,  xaxis3_title='Eccentricity (dva)',  xaxis3_dtick=5,  xaxis4_range=xaxis_range,  xaxis4_title='Eccentricity (dva)',  xaxis4_dtick=5,
                          xaxis5_range=xaxis_range,  xaxis5_title='Eccentricity (dva)',  xaxis5_dtick=5,  xaxis6_range=xaxis_range,  xaxis6_title='Eccentricity (dva)',  xaxis6_dtick=5,
                          xaxis7_range=xaxis_range,  xaxis7_title='Eccentricity (dva)',  xaxis7_dtick=5,  xaxis8_range=xaxis_range,  xaxis8_title='Eccentricity (dva)',  xaxis8_dtick=5,
                          xaxis9_range=xaxis_range,  xaxis9_title='Eccentricity (dva)',  xaxis9_dtick=5,  xaxis10_range=xaxis_range, xaxis10_title='Eccentricity (dva)', xaxis10_dtick=5,
                          xaxis11_range=xaxis_range, xaxis11_title='Eccentricity (dva)', xaxis11_dtick=5, xaxis12_range=xaxis_range, xaxis12_title='Eccentricity (dva)', xaxis12_dtick=5, 
                          template=fig_template, width=fig_width, height=fig_height, margin_l=50, margin_r=20, margin_t=50, margin_b=70,
                          legend_yanchor='top', legend_y=0.99, legend_xanchor='left', legend_x=0.1, legend_bgcolor='rgba(255,255,255,0)', legend_tracegroupgap=0.05)

        # show and save figure
        fig.show(config={"displayModeBar": False})
        fig.write_image("{}/{}_attcmp_r2_ecc{}.pdf".format(tsv_dir, subject, type_analysis))
        fig.write_html("{}/{}_attcmp_r2_ecc{}.html".format(tsv_dir, subject, type_analysis),config={"displayModeBar": False})